In [ ]:
!pip install imutils

In [ ]:
import tensorflow as tf
import keras
from keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import shutil
import cv2
import os

print(tf.version.VERSION)

In [ ]:
from imutils import paths

In [ ]:
dataset_path = './dataset'
log_path = './logs'

In [ ]:
%%bash
rm -rf dataset
mkdir -p dataset/covid
mkdir -p dataset/normal
mkdir -p dataset/pneumonia
mkdir -p logs

In [ ]:
covid_dataset_path='../input/covid-chest-xray'
csvPath = os.path.sep.join([covid_dataset_path, "metadata.csv"])
df = pd.read_csv(csvPath)

for (i, row) in df.iterrows():
    # if (1) the current case is not COVID-19 or (2) this is not
    # a 'PA' view, then ignore the row
    if row["finding"] != "COVID-19" or row["view"] != "PA":
        continue

    imagePath = os.path.sep.join([covid_dataset_path, "images", row["filename"]])

    if not os.path.exists(imagePath):
        continue

    filename = row["filename"].split(os.path.sep)[-1]
    outputPath = os.path.sep.join([f"{dataset_path}/covid", filename])

    shutil.copy2(imagePath, outputPath)

In [ ]:
len(os.listdir('../working/dataset/covid'))

In [ ]:
samples = 141

In [ ]:
pneumonia_dataset_path ='../input/chest-xray-pneumonia/chest_xray'


basePath = os.path.sep.join([pneumonia_dataset_path, "train", "NORMAL"])
imagePaths = list(paths.list_images(basePath))

# randomly sample the image paths
random.seed(42)
random.shuffle(imagePaths)
imagePaths = imagePaths[:samples]

for (i, imagePath) in enumerate(imagePaths):
    filename = imagePath.split(os.path.sep)[-1]
    outputPath = os.path.sep.join([f"{dataset_path}/normal", filename])

    shutil.copy2(imagePath, outputPath)


    
basePath = os.path.sep.join([pneumonia_dataset_path, "train", "PNEUMONIA"])
imagePaths = list(paths.list_images(basePath))

# randomly sample the image paths
random.seed(42)
random.shuffle(imagePaths)
imagePaths = imagePaths[:samples]

for (i, imagePath) in enumerate(imagePaths):
    filename = imagePath.split(os.path.sep)[-1]
    outputPath = os.path.sep.join([f"{dataset_path}/pneumonia", filename])

    shutil.copy2(imagePath, outputPath)

In [ ]:
len(os.listdir('../working/dataset/pneumonia'))

In [ ]:
# !ls ../input/covid-chest-xray/images

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
base_model = tf.keras.models.load_model('/kaggle/input/covid-19-pretrained1/final_cnn_model.h5')

# Show the model architecture
base_model.summary()

In [ ]:
target_names = ['Normal', 'COVID-19', 'Pneumonia']

In [ ]:
from skimage import data, color, io, img_as_float

def get_heatmap(vgg_conv, processed_image, class_idx):
    # we want the activations for the predicted label
    class_output = vgg_conv.output[:, class_idx]
    
    # choose the last conv layer in your model
    last_conv_layer = vgg_conv.get_layer('block5_conv3')
    
    # get the gradients wrt to the last conv layer
    grads = K.gradients(class_output, last_conv_layer.output)[0]
    
    # we pool the gradients over all the axes leaving out the channel dimension
    pooled_grads = K.mean(grads, axis=(0,1,2))
    
    # Define a function that generates the values for the output and gradients
    iterate = K.function([vgg_conv.input], [pooled_grads, last_conv_layer.output[0]])
    
    # get the values
    grads_values, conv_ouput_values = iterate([processed_image])
    
    # iterate over each feature map in your conv output and multiply
    # the gradient values with the conv output values. This gives an 
    # indication of "how important a feature is"
    for i in range(512): # we have 512 features in our last conv layer
        conv_ouput_values[:,:,i] *= grads_values[i]
    
    # create a heatmap
    heatmap = np.mean(conv_ouput_values, axis=-1)
    
    # remove negative values
    heatmap = np.maximum(heatmap, 0)
    
    # normalize
    heatmap /= heatmap.max()
    
    return heatmap

In [ ]:
def print_GradCAM(base_model, path_image):
    # select the sample and read the corresponding image and label
    sample_image = cv2.imread(path_image)
    # pre-process the image
    sample_image = cv2.resize(sample_image, (224,224))
    if sample_image.shape[2] ==1:
                sample_image = np.dstack([sample_image, sample_image, sample_image])
    sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
    sample_image = sample_image.astype(np.float32)/255.
#     sample_label = 1

    #since we pass only one image,we expand dim to include batch size 1
    sample_image_processed = np.expand_dims(sample_image, axis=0)

    # get the label predicted by our original model
    pred_label = np.argmax(base_model.predict(sample_image_processed), axis=-1)[0]
    print(base_model.predict(sample_image_processed))

    # get the heatmap for class activation map(CAM)
    heatmap = get_heatmap(base_model, sample_image_processed, pred_label)
    heatmap = cv2.resize(heatmap, (sample_image.shape[0], sample_image.shape[1]))
    heatmap = heatmap *255
    heatmap = np.clip(heatmap, 0, 255).astype(np.uint8)
    heatmap = 255 - heatmap
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # f,ax = plt.subplots(1,2, figsize=(16,6))
    plt.figure()
    f, ax = plt.subplots(ncols=3, figsize=(16, 6))

    ax[1].imshow(heatmap)
    ax[1].set_title("heatmap")
    ax[1].axis('off')

    #superimpose the heatmap on the image    
    sample_image_hsv = color.rgb2hsv(sample_image)
    heatmap = color.rgb2hsv(heatmap)

    alpha=0.7
    sample_image_hsv[..., 0] = heatmap[..., 0]
    sample_image_hsv[..., 1] = heatmap[..., 1] * alpha

    img_masked = color.hsv2rgb(sample_image_hsv)

    # f,ax = plt.subplots(1,2, figsize=(16,6))
    ax[0].imshow(sample_image)
    ax[0].set_title(f"original image (predicted label: {target_names[pred_label]})")
    ax[0].axis('off')

    ax[2].imshow(img_masked)
    ax[2].set_title("superimposed image")
    ax[2].axis('off')

    plt.show()

In [ ]:
# filename = random.choice(os.listdir('../working/dataset/pneumonia'))
# outputPath = os.path.sep.join([f"../working/dataset/pneumonia", filename])
# outputPath

In [ ]:
# normal
outputPath = '../input/chest-xray-pneumonia/chest_xray/test/NORMAL/IM-0037-0001.jpeg'

In [ ]:
# normal
outputPath = '../input/chest-xray-pneumonia/chest_xray/test/NORMAL/IM-0011-0001-0002.jpeg'

In [ ]:
# pnu
outputPath = '../input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person108_bacteria_507.jpeg'

In [ ]:
# pnu
outputPath = '../input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person120_bacteria_570.jpeg'

In [ ]:
# covid
outputPath = '../input/covid-chest-xray/images/1-s2.0-S0929664620300449-gr2_lrg-a.jpg'

In [ ]:
# covid
outputPath = '../input/covid-chest-xray/images/16654_2_1.jpg'

In [ ]:
# covid
outputPath = '../input/covid-chest-xray/images/16660_1_1.jpg'

In [ ]:
# covid
outputPath = '../input/covid-chest-xray/images/16660_5_1.jpg'

In [ ]:
# covid
outputPath = '../input/covid-chest-xray/images/16664_1_1.jpg'

In [ ]:
# covid -f
outputPath = '../input/covid19-pneumonia-normal-chest-xraypa-dataset/COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset/covid/03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C-1068x1083.jpeg'

In [ ]:
# covid
outputPath = '../input/covid19-pneumonia-normal-chest-xraypa-dataset/COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset/covid/COVID-19 (124).jpg'

In [ ]:
# normal
outputPath = '../input/covid19-pneumonia-normal-chest-xraypa-dataset/COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset/normal/00000005_005.png'

In [ ]:
# normal
outputPath = './covid.png'

In [ ]:
print_GradCAM(base_model, outputPath)

In [ ]:
pneumonia_dataset_path ='../input/chest-xray-pneumonia/chest_xray'


basePath = os.path.sep.join([pneumonia_dataset_path, "train", "NORMAL"])
imagePaths = list(paths.list_images(basePath))

# randomly sample the image paths
random.seed(42)
random.shuffle(imagePaths)
imagePaths = imagePaths[:samples]

for (i, imagePath) in enumerate(imagePaths):
    filename = imagePath.split(os.path.sep)[-1]
    outputPath = os.path.sep.join([f"{dataset_path}/normal", filename])

#     shutil.copy2(imagePath, outputPath)
    


In [ ]:
basePath = os.path.sep.join([pneumonia_dataset_path, "train", "PNEUMONIA"])
imagePaths = list(paths.list_images(basePath))

# randomly sample the image paths
random.seed(42)
random.shuffle(imagePaths)
imagePaths = imagePaths[:samples]

for (i, imagePath) in enumerate(imagePaths):
    filename = imagePath.split(os.path.sep)[-1]
    outputPath = os.path.sep.join([f"{dataset_path}/pneumonia", filename])
    
    print_GradCAM(base_model, imagePath)

#     shutil.copy2(imagePath, outputPath)


In [ ]:
covid_dataset_path='../input/covid-chest-xray'
csvPath = os.path.sep.join([covid_dataset_path, "metadata.csv"])
df = pd.read_csv(csvPath)

for (i, row) in df.iterrows():
    # if (1) the current case is not COVID-19 or (2) this is not
    # a 'PA' view, then ignore the row
    if row["finding"] != "COVID-19" or row["view"] != "PA":
        continue

    imagePath = os.path.sep.join([covid_dataset_path, "images", row["filename"]])

    if not os.path.exists(imagePath):
        continue

    print(imagePath)
    print_GradCAM(base_model, imagePath)
    
#     filename = row["filename"].split(os.path.sep)[-1]
#     outputPath = os.path.sep.join([f"{dataset_path}/covid", filename])

#     shutil.copy2(imagePath, outputPath)

In [ ]:
!pip install wget

In [ ]:
import wget
url = "https://drive.google.com/uc?export=download&id=1dt6yTMEV0kNG7ZUa7rmIZDWH-JGjlzzf"
filename = wget.download(url, out="covid.png")
filename

In [ ]:
# class_to_label_map = {'pneumonia' : 2, 'covid' : 1, 'normal' : 0}

In [ ]:
# dataset_path ='../input/covid-chest-xray/images'
# imagePaths = list(paths.list_images(dataset_path))
# data = []
# labels = []
# for imagePath in imagePaths:
#     label = imagePath.split(os.path.sep)[-2]
#     image = cv2.imread(imagePath)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (224, 224), interpolation = cv2.INTER_AREA)
#     data.append(image)
# #     labels.append(class_to_label_map[label])
    
# data = np.array(data) / 255.0
# # labels = np.array(labels)

In [ ]:
# # select the sample and read the corresponding image and label
# sample_image = cv2.imread('../input/covid-chest-xray/images/1312A392-67A3-4EBF-9319-810CF6DA5EF6.jpeg')
# # pre-process the image
# sample_image = cv2.resize(sample_image, (224,224))
# if sample_image.shape[2] ==1:
#             sample_image = np.dstack([sample_image, sample_image, sample_image])
# sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
# sample_image = sample_image.astype(np.float32)/255.
# sample_label = 1

# #since we pass only one image,we expand dim to include batch size 1
# sample_image_processed = np.expand_dims(data[100], axis=0)
    
# # get the label predicted by our original model
# pred_label = np.argmax(base_model.predict(sample_image_processed), axis=-1)[0]
# print(base_model.predict(sample_image_processed))
    
# # get the heatmap for class activation map(CAM)
# heatmap = get_heatmap(base_model, sample_image_processed, pred_label)
# heatmap = cv2.resize(heatmap, (sample_image.shape[0], sample_image.shape[1]))
# heatmap = heatmap *255
# heatmap = np.clip(heatmap, 0, 255).astype(np.uint8)
# heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# # f,ax = plt.subplots(1,2, figsize=(16,6))
# f, ax = plt.subplots(ncols=3, figsize=(16, 6))

# ax[1].imshow(heatmap)
# ax[1].set_title("heatmap")
# ax[1].axis('off')

# #superimpose the heatmap on the image    
# sample_image_hsv = color.rgb2hsv(sample_image)
# heatmap = color.rgb2hsv(heatmap)

# alpha=0.5
# sample_image_hsv[..., 0] = heatmap[..., 0]
# sample_image_hsv[..., 1] = heatmap[..., 1] * alpha

# img_masked = color.hsv2rgb(sample_image_hsv)

# # f,ax = plt.subplots(1,2, figsize=(16,6))
# ax[0].imshow(sample_image)
# ax[0].set_title(f"True label: {sample_label} \n Predicted label: {pred_label} \n original image")
# ax[0].axis('off')

# ax[2].imshow(img_masked)
# ax[2].set_title("superimposed image")
# ax[2].axis('off')

# plt.show()

In [ ]:
# for i in range(1,50):
#     #since we pass only one image,we expand dim to include batch size 1
#     sample_image_processed = np.expand_dims(data[i], axis=0)

#     # get the label predicted by our original model
#     pred_label = np.argmax(base_model.predict(sample_image_processed), axis=-1)[0]
#     print(base_model.predict(sample_image_processed))

#     # get the heatmap for class activation map(CAM)
#     heatmap = get_heatmap(base_model, sample_image_processed, pred_label)
#     heatmap = cv2.resize(heatmap, (sample_image.shape[0], sample_image.shape[1]))
#     heatmap = heatmap *255
#     heatmap = np.clip(heatmap, 0, 255).astype(np.uint8)
#     heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

#     # f,ax = plt.subplots(1,2, figsize=(16,6))
#     f, ax = plt.subplots(ncols=3, figsize=(16, 6))

#     ax[1].imshow(heatmap)
#     ax[1].set_title("heatmap")
#     ax[1].axis('off')

#     #superimpose the heatmap on the image    
#     sample_image_hsv = color.rgb2hsv(sample_image)
#     heatmap = color.rgb2hsv(heatmap)

#     alpha=0.5
#     sample_image_hsv[..., 0] = heatmap[..., 0]
#     sample_image_hsv[..., 1] = heatmap[..., 1] * alpha

#     img_masked = color.hsv2rgb(sample_image_hsv)

#     # f,ax = plt.subplots(1,2, figsize=(16,6))
#     ax[0].imshow(sample_image)
#     ax[0].set_title(f"True label: {sample_label} \n Predicted label: {pred_label} \n original image")
#     ax[0].axis('off')

#     ax[2].imshow(img_masked)
#     ax[2].set_title("superimposed image")
#     ax[2].axis('off')

#     plt.show()

In [ ]:
# def superimpose(img, cam):
#     """superimpose original image and cam heatmap"""
    
#     heatmap = cv2.resize(cam, (img.shape[1], img.shape[0]))
#     heatmap = np.uint8(255 * heatmap)
#     heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

#     superimposed_img = heatmap * .5 + img * .5
#     superimposed_img = np.minimum(superimposed_img, 255.0).astype(np.uint8)  # scale 0 to 255  
#     superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)
    
#     return img, heatmap, superimposed_img

In [ ]:
# def _plot(model, cam_func, img, cls_true):
#     """plot original image, heatmap from cam and superimpose image"""
    
#     # for cam
#     x = np.expand_dims(img, axis=0)
# #     x = preprocess_input(copy.deepcopy(x))

#     # for superimpose
#     img = np.uint8(img)

#     # cam / superimpose
#     cls_pred, cam = cam_func(model=model, x=x, layer_name=model.layers[-2].name)
#     img, heatmap, superimposed_img = superimpose(img, cam)

#     fig, axs = plt.subplots(ncols=3, figsize=(9, 4))

#     axs[0].imshow(img)
#     axs[0].set_title('original image')
#     axs[0].axis('off')

#     axs[1].imshow(heatmap)
#     axs[1].set_title('heatmap')
#     axs[1].axis('off')

#     axs[2].imshow(superimposed_img)
#     axs[2].set_title('superimposed image')
#     axs[2].axis('off')

#     plt.suptitle('True label: ' + class_to_label[cls_true] + ' / Predicted label : ' + class_to_label[cls_pred])
#     plt.tight_layout()
#     plt.show()

In [ ]:
# ## Grad-CAM function

# def grad_cam(model, x, layer_name):
#     """Grad-CAM function"""
    
#     cls = np.argmax(model.predict(x))
    
#     y_c = model.output[0, cls]
#     conv_output = model.get_layer(layer_name).output
#     grads = K.gradients(y_c, conv_output)[0]

#     # Get outputs and grads
#     gradient_function = K.function([model.input], [conv_output, grads])
#     output, grads_val = gradient_function([x])
#     output, grads_val = output[0, :], grads_val[0, :, :, :]
    
#     weights = np.mean(grads_val, axis=(0, 1)) # Passing through GlobalAveragePooling

#     cam = np.dot(output, weights) # multiply
#     cam = np.maximum(cam, 0)      # Passing through ReLU
#     cam /= np.max(cam)            # scale 0 to 1.0

#     return cls, cam

In [ ]:
# _plot(model=base_model, cam_func=grad_cam, img=data[0], cls_true="Classes[0]")

In [ ]:
# # select the sample and read the corresponding image and label
# sample_image = cv2.imread('../input/covid-chest-xray/images/1.CXRCTThoraximagesofCOVID-19fromSingapore.pdf-001-fig2b.png')
# # pre-process the image
# sample_image = cv2.resize(sample_image, (224,224))
# if sample_image.shape[2] ==1:
#             sample_image = np.dstack([sample_image, sample_image, sample_image])
# sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
# sample_image = sample_image.astype(np.float32)/255.
# sample_label = 1

# #since we pass only one image,we expand dim to include batch size 1
# sample_image_processed = np.expand_dims(sample_image, axis=0)
    
# # get the label predicted by our original model
# pred_label = np.argmax(base_model.predict(sample_image_processed), axis=-1)[0]
# print(base_model.predict(sample_image_processed))
    
# # get the heatmap for class activation map(CAM)
# heatmap = grad_cam(base_model, sample_image_processed, pred_label)
# heatmap = cv2.resize(heatmap, (sample_image.shape[0], sample_image.shape[1]))
# heatmap = heatmap *255
# heatmap = np.clip(heatmap, 0, 255).astype(np.uint8)
# heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# # f,ax = plt.subplots(1,2, figsize=(16,6))
# f, ax = plt.subplots(ncols=3, figsize=(16, 6))

# ax[1].imshow(heatmap)
# ax[1].set_title("heatmap")
# ax[1].axis('off')

# #superimpose the heatmap on the image    
# sample_image_hsv = color.rgb2hsv(sample_image)
# heatmap = color.rgb2hsv(heatmap)

# alpha=0.5
# sample_image_hsv[..., 0] = heatmap[..., 0]
# sample_image_hsv[..., 1] = heatmap[..., 1] * alpha

# img_masked = color.hsv2rgb(sample_image_hsv)

# # f,ax = plt.subplots(1,2, figsize=(16,6))
# ax[0].imshow(sample_image)
# ax[0].set_title(f"True label: {sample_label} \n Predicted label: {pred_label} \n original image")
# ax[0].axis('off')

# ax[2].imshow(img_masked)
# ax[2].set_title("superimposed image")
# ax[2].axis('off')

# plt.show()

In [ ]:
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np
import cv2

class GradCAM:
    def __init__(self, model, classIdx, layerName=None):
        # store the model, the class index used to measure the class
        # activation map, and the layer to be used when visualizing
        # the class activation map
        self.model = model
        self.classIdx = classIdx
        self.layerName = layerName

        # if the layer name is None, attempt to automatically find
        # the target output layer
        if self.layerName is None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):
        # attempt to find the final convolutional layer in the network
        # by looping over the layers of the network in reverse order
        for layer in reversed(self.model.layers):
            # check to see if the layer has a 4D output
            if len(layer.output_shape) == 4:
                return layer.name

        # otherwise, we could not find a 4D layer so the GradCAM
        # algorithm cannot be applied
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")

    def compute_heatmap(self, image, eps=1e-8):
        # construct our gradient model by supplying (1) the inputs
        # to our pre-trained model, (2) the output of the (presumably)
        # final 4D layer in the network, and (3) the output of the
        # softmax activations from the model
        gradModel = Model(
            inputs=[self.model.inputs],
            outputs=[self.model.get_layer(self.layerName).output, 
                self.model.output])

        # record operations for automatic differentiation
        with tf.GradientTape() as tape:
            # cast the image tensor to a float-32 data type, pass the
            # image through the gradient model, and grab the loss
            # associated with the specific class index
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            loss = predictions[:, self.classIdx]

        # use automatic differentiation to compute the gradients
        grads = tape.gradient(loss, convOutputs)

        # compute the guided gradients
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads

        # the convolution and guided gradients have a batch dimension
        # (which we don't need) so let's grab the volume itself and
        # discard the batch
        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]

        # compute the average of the gradient values, and using them
        # as weights, compute the ponderation of the filters with
        # respect to the weights
        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

        # grab the spatial dimensions of the input image and resize
        # the output class activation map to match the input image
        # dimensions
        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))

        # normalize the heatmap such that all values lie in the range
        # [0, 1], scale the resulting values to the range [0, 255],
        # and then convert to an unsigned 8-bit integer
        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")

        # return the resulting heatmap to the calling function
        return heatmap

    def overlay_heatmap(self, heatmap, image, alpha=0.5,
        colormap=cv2.COLORMAP_VIRIDIS):
        # apply the supplied color map to the heatmap and then
        # overlay the heatmap on the input image
        heatmap = cv2.applyColorMap(heatmap, colormap)
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)

        # return a 2-tuple of the color mapped heatmap and the output,
        # overlaid image
        return (heatmap, output)

In [ ]:
# from pyimagesearch.gradcam import GradCAM
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import imagenet_utils
import numpy as np
import argparse
import imutils
import cv2

# # construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,
# 	help="path to the input image")
# ap.add_argument("-m", "--model", type=str, default="vgg",
# 	choices=("vgg", "resnet"),
# 	help="model to be used")
# args = vars(ap.parse_args())

# # initialize the model to be VGG16
# Model = VGG16

# # check to see if we are using ResNet
# if args["model"] == "resnet":
# 	Model = ResNet50

# # load the pre-trained CNN from disk
# print("[INFO] loading model...")
# model = Model(weights="imagenet")

# load the original image from disk (in OpenCV format) and then
# resize the image to its target dimensions
imgpath = '../input/covid19-pneumonia-normal-chest-xraypa-dataset/COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset/covid/01E392EE-69F9-4E33-BFCE-E5C968654078-768x572.jpeg'
orig = cv2.imread(imgpath)
resized = cv2.resize(orig, (224, 224))

# load the input image from disk (in Keras/TensorFlow format) and
# preprocess it
image = load_img(imgpath, target_size=(224, 224))
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = imagenet_utils.preprocess_input(image)

# use the network to make predictions on the input imag and find
# the class label index with the largest corresponding probability
preds = base_model.predict(image)
i = np.argmax(preds[0])

# decode the ImageNet predictions to obtain the human-readable label
decoded = imagenet_utils.decode_predictions(preds)
(imagenetID, label, prob) = decoded[0][0]
label = "{}: {:.2f}%".format(label, prob * 100)
print("[INFO] {}".format(label))

# initialize our gradient class activation map and build the heatmap
cam = GradCAM(base_model, i, 'block5_conv3')
heatmap = cam.compute_heatmap(image)

# resize the resulting heatmap to the original input image dimensions
# and then overlay heatmap on top of the image
heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
(heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)

# draw the predicted label on the output image
cv2.rectangle(output, (0, 0), (340, 40), (0, 0, 0), -1)
cv2.putText(output, label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

# display the original image and resulting heatmap and output image
# to our screen
output = np.vstack([orig, heatmap, output])
output = imutils.resize(output, height=700)
# cv2.imshow("Output", output)
# cv2.waitKey(0)
plt.imshow(output)

In [ ]:

# from tensorflow.keras.preprocessing.image import img_to_array
# from tensorflow.keras.preprocessing.image import load_img
# from tensorflow.keras.applications import imagenet_utils
# import numpy as np
# import argparse
# import imutils
# import matplotlib.pyplot as plt

In [ ]:
orig = cv2.imread('../input/covid-chest-xray/images/1-s2.0-S0140673620303706-fx1_lrg.jpg')
resized = cv2.resize(orig, (224, 224))

# load the input image from disk (in Keras/TensorFlow format) and
# preprocess it
image = load_img('../input/covid-chest-xray/images/1-s2.0-S0140673620303706-fx1_lrg.jpg', target_size=(224, 224))
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = imagenet_utils.preprocess_input(image)

# use the network to make predictions on the input imag and find
# the class label index with the largest corresponding probability
preds = base_model.predict(image)
print(preds)
i = np.argmax(preds[0])
print(i)

# # decode the ImageNet predictions to obtain the human-readable label
# decoded = imagenet_utils.decode_predictions(preds)
# (imagenetID, label, prob) = decoded[0][0]
# label = "{}: {:.2f}%".format(label, prob * 100)
# print("[INFO] {}".format(label))

label ="hello"

# initialize our gradient class activation map and build the heatmap
cam = GradCAM(base_model, i)
heatmap = cam.compute_heatmap(image)

# resize the resulting heatmap to the original input image dimensions
# and then overlay heatmap on top of the image
heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
(heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)

# draw the predicted label on the output image
cv2.rectangle(output, (0, 0), (340, 40), (0, 0, 0), -1)
cv2.putText(output, label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX,
	0.8, (255, 255, 255), 2)

# display the original image and resulting heatmap and output image
# to our screen
output = np.vstack([orig, heatmap, output])
output = imutils.resize(output, height=700)
# cv2.imwrite("Output.png", output)
plt.imshow(output)